# Initialization

## modules

In [2]:
from case_study.integrations.database import  connections
from case_study.integrations.duckdb import DuckDBIntegrator
from case_study.ai_join_advisor import SmartJoinBuilder
from case_study.jupyter_ui import select_databases_widget,select_tables_widget

2025-04-15 17:58:39,504 - database-queries - INFO - Initializing ConnectionConfig with path: config.yaml
2025-04-15 17:58:39,622 - database-queries - INFO - Loading configuration from config.yaml
2025-04-15 17:58:39,776 - database-queries - INFO - Processing connection configuration for: postgres_datamancers
2025-04-15 17:58:39,916 - database-queries - INFO - Successfully configured connection: postgres_datamancers
2025-04-15 17:58:40,077 - database-queries - INFO - Processing connection configuration for: datamancers
2025-04-15 17:58:40,224 - database-queries - INFO - Successfully configured connection: datamancers
2025-04-15 17:58:40,386 - database-queries - INFO - ConnectionConfig initialization completed


## Required objects
- SmartJoinBuilder() - class that suggest join between two selected tables
- DuckDbIntegrator() - class for querying across multiple (types) of databases within single query  

In [3]:
builder = SmartJoinBuilder()
query_engine = DuckDBIntegrator()

2025-04-15 17:58:42,328 - join-advisor - INFO - Initializing SmartJoinBuilder with config path: config.yaml
2025-04-15 17:58:42,721 - duckdb-integrator - INFO - Initializing DuckDB integrator with config path: config.yaml
2025-04-15 17:58:42,871 - duckdb-integrator - INFO - Using DuckDB database path: data/duckdb_logs.db
2025-04-15 17:58:43,024 - duckdb-integrator - INFO - Ensured database directory exists: data
2025-04-15 17:58:43,172 - duckdb-integrator - INFO - DuckDB connection established
2025-04-15 17:58:43,339 - database-queries - INFO - Initializing ConnectionConfig with path: config.yaml
2025-04-15 17:58:43,488 - database-queries - INFO - Loading configuration from config.yaml
2025-04-15 17:58:43,652 - database-queries - INFO - Processing connection configuration for: postgres_datamancers
2025-04-15 17:58:43,779 - database-queries - INFO - Successfully configured connection: postgres_datamancers
2025-04-15 17:58:43,922 - database-queries - INFO - Processing connection configur

# Workflow
1. select databases to connect from your configuration file 
2. Smart Join Suggest (optional): do you want to join two tables across your connected databases but don't know how? Select them and let the ai give you the suggestion!
3. Querying engine: Run your query across databases! You can query any data across your connections, just wrap the table reference into "DB type function" (e.g. BIGQUERY(project-id.dataset_id.table_id)) and let the magic flow! You can also join tables across databases!

In [ ]:
select_databases_widget(connections,builder)

2025-04-15 17:50:01,140 - database-queries - INFO - Available connections: ['postgres_datamancers', 'datamancers']
2025-04-15 17:50:01,291 - database-queries - INFO - Retrieving connection info for: postgres_datamancers
2025-04-15 17:50:01,455 - database-queries - INFO - Retrieving connection info for: datamancers


(SelectMultiple(description='Select Databases:', layout=Layout(width='auto'), options=(('postgres_datamancers (postgres)', 'postgres_datamancers'), ('datamancers (bigquery)', 'datamancers')), style=DescriptionStyle(description_width='initial'), value=()),)

In [10]:
select_tables_widget(builder)

{'table1': None, 'table2': None}

In [7]:
query="""WITH
    bq_data AS (
        SELECT client_id, project_id, resource_id FROM BIGQUERY('datamancers.dbt.L1_POSTGRES__JOBS__planned_cleaned')
    ), pg_data AS (
        SELECT * FROM POSTGRES('public.project_resources_pricings'))
        SELECT * FROM pg_data LEFT JOIN bq_data ON pg_data.project_id = bq_data.project_id"""

data=query_engine.run_query(query)

/Users/vitmrnavek/projects/datamancers/clients/rohlik/case_study/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


2025-04-15 17:51:50,491 - duckdb-integrator - INFO - Logging query performance for source: bigquery
2025-04-15 17:51:50,626 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-15 17:51:51,172 - duckdb-integrator - ERROR - Error parsing execution plan: Expecting value: line 1 column 1 (char 0)
2025-04-15 17:51:51,288 - duckdb-integrator - INFO - Logging query performance for source: postgres
2025-04-15 17:51:51,475 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-15 17:51:52,037 - duckdb-integrator - INFO - Logging query performance for source: duckdb
2025-04-15 17:51:52,155 - duckdb-integrator - INFO - Query performance logged successfully


In [4]:

logs = builder.get_performance_logs()
logs

2025-04-15 17:58:58,598 - join-advisor - INFO - Retrieving performance logs (start_time=None, end_time=None)
2025-04-15 17:58:58,746 - duckdb-integrator - INFO - Retrieving query logs (start_time=None, end_time=None)
2025-04-15 17:58:58,907 - duckdb-integrator - INFO - Retrieved 18 query log entries


,query_id,timestamp,source,query_text,execution_time_ms,rows_affected,execution_plan,error
0,6584198339106515449,2025-04-15 17:58:17.642999,duckdb,WITH\n bq_data AS (\n SELECT client_...,37425.123,1985,"{""explain_analyze"": [""analyzed_plan""]}",None
1,5204762967287917805,2025-04-15 17:58:06.062143,postgres,SELECT * FROM public.project_resources_pricings,1636.492,25,"{""result_set_size"": 0, ""cumulative_rows_scanne...",None
2,5004238421755160957,2025-04-15 17:57:23.424100,bigquery,SELECT * FROM `datamancers.dbt.L1_POSTGRES__JO...,89.000,3331,"{""total_bytes_processed"": 0, ""total_bytes_bill...",None
3,7459996599962420906,2025-04-15 17:55:13.644479,bigquery,SELECT * FROM `datamancers.dbt.L1_POSTGRES__JO...,107.000,3331,"{""total_bytes_processed"": 0, ""total_bytes_bill...",None
4,4730250147978338446,2025-04-15 17:51:52.037499,duckdb,WITH\n bq_data AS (\n SELECT client_...,925.702,1985,"{""explain_analyze"": [""analyzed_plan""]}",None
5,2683042214955702766,2025-04-15 17:51:51.289199,postgres,SELECT * FROM public.project_resources_pricings,148.229,25,{},Expecting value: line 1 column 1 (char 0)
6,6911884955644571003,2025-04-15 17:51:50.492404,bigquery,SELECT * FROM `datamancers.dbt.L1_POSTGRES__JO...,109.000,3331,"{""total_bytes_processed"": 0, ""total_bytes_bill...",None
7,6357741692435740746,2025-04-15 17:50:50.020964,duckdb,WITH\n bq_data AS (\n SELECT client_...,946.634,1985,"{""explain_analyze"": [""analyzed_plan""]}",None
8,7292616554699960895,2025-04-15 17:50:49.294337,postgres,SELECT * FROM public.project_resources_pricings,142.953,25,{},Expecting value: line 1 column 1 (char 0)
9,5920777246850019964,2025-04-15 17:50:48.134909,bigquery,SELECT * FROM `datamancers.dbt.L1_POSTGRES__JO...,87.000,3331,"{""total_bytes_processed"": 0, ""total_bytes_bill...",None
